# Naver max-rating cralwer

네이버에 프로그램 이름을 검색했을 때 나오는 최고 시청률(닐슨)을 긁어오는 크롤러

In [6]:
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import os
import pandas as pd

## 1. Data load

In [27]:
drama = pd.read_csv("info_crawling_all_clear2.csv")

In [28]:
drama = drama.sort_values(['name']).reset_index()

In [29]:
#결측값 처리된 파일인지 확인하기
drama.isnull().sum()

index      0
name       0
country    0
channel    0
period     0
count      0
time       0
dtype: int64

In [30]:
name = drama['name']

In [32]:
drama = drama[['name', 'country', 'channel', 'period', 'count', 'time']]

In [33]:
drama

,name,country,channel,period,count,time
0,(아는 건 별로 없지만) 가족입니다,대한민국,tvN,2020년 6월 1일 ~ 2020년 7월 21일,16부작,"매주 월, 화 밤 9시"
1,17세의 조건,대한민국,SBS,2019년 8월 5일 ~ 2019년 8월 6일,2부작,"매주 월, 화 밤 10:00 ~ 11:10"
2,20세기 소년소녀,대한민국,MBC,2017년 10월 9일 ~ 2017년 11월 28일,32부작,"매주 월, 화 밤 10:00 ~ 11:10 (1회~28회)\n매주 월, 화 저녁 8..."
3,365 : 운명을 거스르는 1년,대한민국,MBC TV,2020년 3월 23일 ~ 2020년 4월 28일,24부작,"매주 월, 화 밤 8시 55분 ~ 10시 5분"
4,38 사기동대,대한민국,OCN,2016년 6월 17일 ~ 2016년 8월 6일,16부작,금 · 토요일 밤 11시
...,...,...,...,...,...,...
498,훈장 오순남,대한민국,MBC,2017년 4월 24일 ~ 2017년 10월 20일,129부작,월 ~ 금 오전 7시 50분 ~ 8시 30분
499,흉부외과 - 심장을 훔친 의사들,대한민국,SBS,2018년 9월 27일 ~ 2018년 11월 15일,32부작,"매주 수, 목 밤 10:00 ~ 11:10"
500,흑기사,대한민국,KBS 2TV,2017년 12월 6일 ~ 2018년 2월 8일,20부작,"매주 수, 목 밤 10:00 ~ 11:05"
501,힘쎈여자 도봉순,대한민국,JTBC,2017년 2월 24일 ~ 2017년 4월 15일,16부작,"매주 금요일, 토요일 밤 11시"


## 2. Preprocessing for crawling

ex) 도깨비 (2019년 드라마)   
처럼 프로그램 이름 다음에 ( )안에 설명이 있는 경우가 있어서, 이를  지우는 작업을 진행함

In [38]:
name_bf = drama['name']

In [39]:
len(name_bf)

503

In [51]:
for i in range(len(name_bf)):
    if name_bf[i].find('(', 3) > 0:
        st = name_bf[i].find('(', 3)
        ed = name_bf[i].find(')')
        print(name_bf[i])
        print(name_bf[i][:st])

In [50]:
for i in range(len(name_bf)):
    if name_bf[i].find('(', 3) > 0:
        st = name_bf[i].find('(', 3)
        ed = name_bf[i].find(')')
        name_bf[i] = name_bf[i][:st]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
name_bf

0      (아는 건 별로 없지만) 가족입니다
1                  17세의 조건
2                20세기 소년소녀
3        365 : 운명을 거스르는 1년
4                  38 사기동대
              ...         
498                 훈장 오순남
499      흉부외과 - 심장을 훔친 의사들
500                    흑기사
501               힘쎈여자 도봉순
502             힙합왕 - 나스나길
Name: name, Length: 503, dtype: object

## 3. Naver max-rating cralwer

In [84]:
name = drama['name']
max_rating = []
source = []

for i in range(len(name)):
    max_rating.append("") 
    source.append("")

In [85]:
print(len(name), len(max_rating), len(source))

503 503 503


In [86]:
# driver
driver = webdriver.Chrome(executable_path="C:\chromedriver\chromedriver.exe")

for i in range(0, len(name)):
    driver.get("https://www.naver.com/")
    elem = driver.find_element_by_id('query')
    elem.send_keys(name[i])
    elem.submit()
    
    ##### crawling #####
    print("-------" + str(i) + "행을 크롤링중입니다.--------")
    print("< " + name[i] + " >")
    
    try:
        info = driver.find_element_by_class_name('detail_info')
        info_box = info.find_element_by_class_name('data_box')
        rating = info_box.find_element_by_class_name('value')
        
    except NoSuchElementException:
        print("Nothing")
        continue
    
    print(rating.text)
    print(info_box.text)
    
    max_rating[i] = rating.text
    source[i] = info_box.text
        
#         tbody = table.find_element_by_tag_name("tbody")
#         span = tbody.find_elements_by_tag_name("span")
    
#     except NoSuchElementException:
#         print("Nothing")
#         continue

-------0행을 크롤링중입니다.--------
< (아는 건 별로 없지만) 가족입니다 >
5.4%
5.4%닐슨코리아
-------1행을 크롤링중입니다.--------
< 17세의 조건 >
3.0%
3.0%닐슨코리아
-------2행을 크롤링중입니다.--------
< 20세기 소년소녀 >
4.3%
4.3%닐슨코리아
-------3행을 크롤링중입니다.--------
< 365 : 운명을 거스르는 1년 >
5.1%
5.1%닐슨코리아
-------4행을 크롤링중입니다.--------
< 38 사기동대 >
4.6%
4.6%닐슨코리아
-------5행을 크롤링중입니다.--------
< 60일, 지정생존자 >
6.2%
6.2%닐슨코리아
-------6행을 크롤링중입니다.--------
< 7일의 왕비 >
7.7%
7.7%닐슨코리아
-------7행을 크롤링중입니다.--------
< 99억의 여자 >
11.6%
11.6%닐슨코리아
-------8행을 크롤링중입니다.--------
< EXIT >
Nothing
-------9행을 크롤링중입니다.--------
< SKY 캐슬 >
23.8%
23.8%닐슨코리아
-------10행을 크롤링중입니다.--------
< VIP >
15.9%
15.9%닐슨코리아
-------11행을 크롤링중입니다.--------
< W >
13.8%
13.8%닐슨코리아
-------12행을 크롤링중입니다.--------
< 가면 >
13.6%
13.6%닐슨코리아
-------13행을 크롤링중입니다.--------
< 가족을 지켜라 >
28.2%
28.2%닐슨코리아
-------14행을 크롤링중입니다.--------
< 가화만사성 >
20.4%
20.4%닐슨코리아
-------15행을 크롤링중입니다.--------
< 간택 - 여인들의 전쟁 >
6.3%
6.3%닐슨코리아
-------16행을 크롤링중입니다.--------
< 강남 스캔들 >
7.9%
7.9%닐슨코리아
-------17행을 크롤링중입니다.--------
< 강덕순 애정 변천사 

## 4. Data save

In [93]:
drama['max_rating'] = pd.DataFrame(max_rating)
drama['source'] = pd.DataFrame(source)

In [92]:
for i in range(len(source)):
    st = source[i].find("%")
    source[i] = source[i][st+1:]

In [94]:
drama

,name,country,channel,period,count,time,max_rating,source
0,(아는 건 별로 없지만) 가족입니다,대한민국,tvN,2020년 6월 1일 ~ 2020년 7월 21일,16부작,"매주 월, 화 밤 9시",5.4%,닐슨코리아
1,17세의 조건,대한민국,SBS,2019년 8월 5일 ~ 2019년 8월 6일,2부작,"매주 월, 화 밤 10:00 ~ 11:10",3.0%,닐슨코리아
2,20세기 소년소녀,대한민국,MBC,2017년 10월 9일 ~ 2017년 11월 28일,32부작,"매주 월, 화 밤 10:00 ~ 11:10 (1회~28회)\n매주 월, 화 저녁 8...",4.3%,닐슨코리아
3,365 : 운명을 거스르는 1년,대한민국,MBC TV,2020년 3월 23일 ~ 2020년 4월 28일,24부작,"매주 월, 화 밤 8시 55분 ~ 10시 5분",5.1%,닐슨코리아
4,38 사기동대,대한민국,OCN,2016년 6월 17일 ~ 2016년 8월 6일,16부작,금 · 토요일 밤 11시,4.6%,닐슨코리아
...,...,...,...,...,...,...,...,...
498,훈장 오순남,대한민국,MBC,2017년 4월 24일 ~ 2017년 10월 20일,129부작,월 ~ 금 오전 7시 50분 ~ 8시 30분,11.6%,닐슨코리아
499,흉부외과 - 심장을 훔친 의사들,대한민국,SBS,2018년 9월 27일 ~ 2018년 11월 15일,32부작,"매주 수, 목 밤 10:00 ~ 11:10",8.8%,닐슨코리아
500,흑기사,대한민국,KBS 2TV,2017년 12월 6일 ~ 2018년 2월 8일,20부작,"매주 수, 목 밤 10:00 ~ 11:05",13.9%,닐슨코리아
501,힘쎈여자 도봉순,대한민국,JTBC,2017년 2월 24일 ~ 2017년 4월 15일,16부작,"매주 금요일, 토요일 밤 11시",9.7%,닐슨코리아


In [96]:
drama.to_csv("drama_crawling_max_rating.csv", index=False, encoding='utf-8-sig')

<hr>

##  << 드라마 카테고리 분류 (period, time을 이용) >>

예시 케이스

1. 미니시리즈
월, 화 - 20부작 미만 
수, 목 - 20부작 미만
월 / 화 / 수 / 목 - 20부작 미만 

2. 아침드라마
매주 평일 아침 
오전 키워드가 들어간 경우

3. 일일드라마
매주 평일 저녁

4. 주말드라마
토, 일
토
일

## 1. Data load

In [154]:
drama_cate = pd.read_csv("drama_crawling_max_rating_2.csv")

In [155]:
drama_cate

,name,country,channel,period,count,time,max_rating,source
0,시간이 멈추는 그때,대한민국,KBS W,2018년 10월 24일 ~ 2018년 11월 29일,12부작,"수요일, 목요일 밤 11시",0.10%,닐슨코리아
1,그남자 오수,대한민국,OCN,2018년 3월 5일 ~ 2018년 4월 24일,16부작,"월요일, 화요일 밤 9시",0.40%,닐슨코리아
2,애타는 로맨스,대한민국,"OCN, 라이프타임",2017년 4월 17일 ~ 2017년 5월 30일\n2019년 5월 15일 ~ 20...,13부작\n14부작(일본DVD 편집본),"월요일, 화요일 밤 9시\n수요일, 목요일 오후 3시",0.50%,닐슨코리아
3,열두밤,대한민국,채널A,2018년 10월 5일 (스페셜 방송)\n2018년 10월 12일 ~ 2018년 1...,12부작,금요일 밤 11시,0.60%,닐슨코리아
4,커피야 부탁해,대한민국,채널A,2018년 12월 1일 ~ 2018년 12월 30일,12부작,"토요일, 일요일 오후 7시 40분 ~",0.60%,닐슨코리아
...,...,...,...,...,...,...,...,...
494,처음이라서,대한민국,"On Style, OLIVE",2015년 10월 7일 ~2015년 11월 25일,8부작,매주 수요일 밤 11시,조회불가,닐슨코리아
495,파수꾼,대한민국,MBC,2017년 5월 22일 ~ 2017년 7월 11일,32부작,"매주 월, 화 밤 10:00 ~ 11:00",10.20%,닐슨코리아
496,프리스트,대한민국,OCN,2018년 11월 24일 ~ 2019년 1월 20일,16부작,"토요일, 일요일 밤 10시 20분",2.50%,닐슨코리아
497,플레이어,대한민국,OCN,2018년 9월 29일 ~ 2018년 11월 11일,14부작,"토요일, 일요일 밤 10시 20분",5.80%,닐슨코리아


## 2. Rule-based categorization

In [169]:
name = drama_cate['name']
time = drama_cate['time']
count = drama_cate['count']
weekday = []
weekend = []
morning = []
mini = []
etc = []
count2 = []

for i in range(len(name)):
    weekday.append(0) 
    weekend.append(0) 
    morning.append(0) 
    mini.append(0) 
    etc.append(0)
    count2.append("")

In [170]:
for i in range(len(name)):
    count2[i] = count[i][:count[i].find("부작")]

In [171]:
count2 = list(map(int, count2))

In [172]:
# mini 시리즈인 경우 할당 :: 조건 (20부작 이내면서 월/화/수/목 중 하나가 들어가 있는 경우)
for i in range(len(name)):
    if count2[i] <= 20 and ("월" in time[i] or "화" in time[i] or "수" in time[i] or "목" in time[i]):
        print(name[i], count[i], time[i])
        mini[i] = 1

시간이 멈추는 그때 12부작 수요일, 목요일 밤 11시
그남자 오수 16부작 월요일, 화요일 밤 9시
애타는 로맨스 13부작
14부작(일본DVD 편집본) 월요일, 화요일 밤 9시
수요일, 목요일 오후 3시
로스타임 라이프 : 더 라스트 찬스 2부작 수요일, 목요일 밤 11시
달리는 조사관 14부작 매주 수요일, 목요일 밤 11시
최고의 치킨 12부작 수요일, 목요일 밤 11시
시를 잊은 그대에게 16부작 매주 월 · 화요일 밤 9시 30분
야식남녀 12부작 매주 월요일, 화요일 밤 9시 30분 ~ 10시 44분
풍선껌 16부작 월요일, 화요일 밤 11시
그녀는 거짓말을 너무 사랑해 16부작 매주 월, 화 오후 11시
대장금이 보고 있다 16부작 목요일
리치맨 16부작 매주 수요일, 목요일 밤 11시
멈추고 싶은 순간 : 어바웃 타임 16부작 매주 월 · 화요일 밤 9시 30분
으라차차 와이키키 20부작 매주 월요일, 화요일 밤 11시
으라차차 와이키키 2 16부작 매주 월요일, 화요일 밤 9시 30분
마성의 기쁨 16부작 수요일, 목요일 밤 11시
마녀의 사랑 12부작 수요일, 목요일 밤 11시
그냥 사랑하는 사이 16부작 매주 월요일, 화요일 밤 11시
반의 반 12부작 매주 월, 화 밤 9시
미치겠다, 너땜에! 4부작 매주 월, 화 밤 10:00 ~ 11:10
신분을 숨겨라 16부작 월요일, 화요일 밤 11시
빙의 16부작 매주 수요일, 목요일 밤 11시
계약우정  8부작 매주 월요일, 화요일 밤 10:00 ~ 11:10
날씨가 좋으면 찾아가겠어요 16부작 매주 월, 화 밤 9시 30분 ~ 10시 42분
사이코메트리 그녀석 16부작 월, 화
생일편지 4부작 수요일 밤 10:00 ~ 11:10, 목요일 밤 9:55 ~ 11:10
설렘주의보 16부작 수요일, 목요일 밤 11시
강덕순 애정 변천사 1부작 목요일 밤 10:00 ~ 11:05
써클: 이어진 두 세계 12부작 매주 월요일, 화요일 밤 11시
17세의 조건 2부작 매주 월, 화 밤 10:00 ~ 11:

In [220]:
# 기타 (금,토이면서 16부작인 것들) // 월, 화, 수, 목 이면서 20부작 초과인 애들
for i in range(len(name)):
    if count2[i] <= 60 and ("금" in time[i] and "토" in time[i]):
        print(name[i], count[i], time[i])
        etc[i] = 1
    if count2[i] > 20 and count2[i] <= 80 and ("월" in time[i] or "화" in time[i] or "수" in time[i] or "목" in time[i]):
        print(name[i], count[i], time[i])
        etc[i] = 1

유별나! 문셰프 16부작 매주 금, 토 오후 10시50분 ~ 오전 12시10분
터치  16부작 매주 금요일, 토요일 밤 10시 50분 ~ 12시 10분
솔로몬의 위증 12부작 매주 금요일, 토요일 밤 8시 30분
멜로가 체질 16부작 + 스페셜 1부작 매주 금요일, 토요일 밤 10시 50분 ~ 12시 20분
사랑하는 은동아 16부작 매주 금요일, 토요일 밤 8시 40분 (1회 ~ 14회)
매주 금요일, 토요일 밤 8시 30분 (15회 ~ 16회)
어서와 24부작 매주 수요일, 목요일 밤 10:00 ~ 11:10
평일 오후 세시의 연인 16부작 매주 금요일, 토요일 밤 11시 ~ 12시 10분
라스트 16부작 매주 금요일, 토요일 밤 8시 30분
안투라지  16부작 매주 금 · 토요일 밤 11시
더 패키지 12부작 매주 금요일, 토요일 밤 11시
판타스틱 16부작 매주 금요일, 토요일 밤 8시 30분
시카고 타자기 16부작 + 스페셜 2부작 매주 금 · 토요일 밤 8시 (1~8회)
밤 8시 30분 (9~16회)
마녀보감 20부작 매주 금요일, 토요일 밤 8시 30분
욱씨남정기 16부작 매주 금요일, 토요일 밤 8시 30분
이번 주 아내가 바람을 핍니다 12부작 매주 금요일, 토요일 밤 8시 30분
제3의 매력 16부작 매주 금요일, 토요일 밤 11시
절대 그이 36부작 매주 수, 목 밤 10:00 ~ 11:10
위대한 유혹자 32부작 매주 월, 화 밤 10:00 ~ 11:10
스케치 16부작 매주 금요일, 토요일 밤 11시
기억 16부작 매주 금 · 토요일 밤 8시 30분
내일 그대와 16부작 매주 금 · 토요일 밤 8시
신데렐라와 네 명의 기사 16부작 매주 금 · 토요일 밤 11시 15분
너의 노래를 들려줘 32부작 매주 월요일, 화요일 밤 10:00 ~ 11:10
배드파파 32부작 매주 월요일, 화요일 밤 10:00 ~ 11:10
언터처블  16부작 매주 금요일, 토요일 밤 11시
죽어도 좋아 32부작 + 스페셜 1부작 매주 수요일, 목요일 밤 10:00 

In [221]:
# 아침 드라마인 경우 
for i in range(len(name)):
    if "아침" in time[i]:
        print(name[i], count[i], time[i])
        morning[i] = 1

찬란한 내 인생 120부작 매주 평일 저녁 7:20 ~ 7:55 (본방송)
매주 평일 아침 8:50 ~ 9:30 (재방송)
모두 다 쿵따리 99부작 매주 평일 아침 7:50 ~ 8:30
나쁜 사랑 129부작 매주 평일 아침 7:50 ~ 8:30
엄마가 바람났다 120부작 매주 평일 아침 8:35 ~ 9:10
수상한 장모 123부작 매주 평일 아침 8:35 ~ 9:10
맛 좀 보실래요 124부작 매주 평일 아침 8:35 ~ 9:10
결혼계약 16부작 매주 주말 밤 10:00 ~ 11:15 -> 매주 월요일 ~ 화요일, 금요일 아침 10:45 ~ 낮 12:00


In [222]:
# 주말 드라마인 경우
for i in range(len(name)):
    if ("토" in time[i] or "일요일" in time[i]) and "금" not in time[i] :
        print(name[i], count[i], time[i])
        weekend[i] = 1
    
    if "주말" in time[i]:
        print(name[i], count[i], time[i])
        weekend[i] = 1

커피야 부탁해 12부작 토요일, 일요일 오후 7시 40분 ~
뱀파이어 탐정 12부작 일요일 밤 11시
미스트리스  12부작 매주 토요일, 일요일 밤 10시
조선생존기 16부작 매주 토요일, 일요일 밤 10시 50분
동네의 영웅 16부작 토요일, 일요일 밤 11시
어쩌다 가족 2부작 매주 일요일 오후 7시 55분
모두의 거짓말 16부작 토요일, 일요일 밤 10시 30분 ~ 11시 50분
송곳 12부작 토요일 일요일 밤 9시 40분
레버리지: 사기조작단 16부작 매주 일요일 밤 10시 (2회 연속방송)
안단테 16부작 매주 일요일 오전 10:10 ~ 11:05
킬잇 12부작 토요일, 일요일 밤 10시 20분
날 녹여주오 16부작 매주 토요일 밤 9시 10분 ~ 10시 23분
매주 일요일 밤 9시 10분 ~ 10시 21분
구해줘 16부작 토요일, 일요일 밤 10시 20분
바벨 16부작 + 스페셜1부작 토요일, 일요일 밤 10시 50분
변혁의 사랑 16부작 매주 토 · 일요일 밤 9시
루갈 16부작 매주 토요일, 일요일 밤 10시 50분
작은 신의 아이들 16부작 토요일, 일요일 밤 10시
타인은 지옥이다 10부작 토요일 일요일 밤 10시 30분 ~ 11시 50분
트랩 7부작 토요일, 일요일 밤 10시 20분
번외수사 12부작 매주 토요일,일요일 밤 10시 50분
본 대로 말하라 16부작 토요일, 일요일 밤 10시 50분 ~ 12시 10분
나쁜 녀석들 : 악의 도시 16부작 토요일, 일요일 밤 10시 20분
화양연화 - 삶이 꽃이 되는 순간 16부작 매주 토요일 밤 9시 10분 ~ 10시 30분
매주 일요일 밤 9시 10분 ~ 10시 35분
보이스 16부작 매주 토 · 일요일 밤 10시
보이스 3 16부작 토요일, 일요일 밤 10시 20분
이혼변호사는 연애중 18부작 토, 일 밤 10시 ~ 11시 10분
나인룸 16부작 토, 일
간택 - 여인들의 전쟁 16부작 매주 토요일, 일요일 오후 11시 ~ 오전 0시
바람과 구름과 비  21부작 매주 토요일, 일요일 오후 10

In [223]:
# 일일 드라마인 경우 
for i in range(len(name)):
    if count2[i] >= 80 or "매주 평일 저녁" in time[i]:
        print(name[i], count[i], time[i])
        weekday[i] = 1

울지 않는 새 100부작 월요일 ~ 목요일 오전 9:40 ~ 10:30
찬란한 내 인생 120부작 매주 평일 저녁 7:20 ~ 7:55 (본방송)
매주 평일 아침 8:50 ~ 9:30 (재방송)
모두 다 쿵따리 99부작 매주 평일 아침 7:50 ~ 8:30
나쁜 사랑 129부작 매주 평일 아침 7:50 ~ 8:30
엄마가 바람났다 120부작 매주 평일 아침 8:35 ~ 9:10
수상한 장모 123부작 매주 평일 아침 8:35 ~ 9:10
강남 스캔들 123부작 월 ~ 금 오전 8시 40분 ~ 9시 10분
사랑은 방울방울 120부작 월 ~ 금 저녁 7시 20분 ~ 8시
역류 119부작 매주 평일 오전 7:50 ~ 8:30
용왕님 보우하사 121부작 매주 평일 저녁 7:10 ~ 7:55 [1회~5회]
매주 평일 저녁 7:15 ~ 7:55 [6회~40회]
매주 평일 저녁 6:50 ~ 7:30 [41회~121회]
차달래 부인의 사랑 100부작 매주 평일 오전 9:00 ~ 9:40
맛 좀 보실래요 124부작 매주 평일 아침 8:35 ~ 9:10
나도 엄마야 124부작 월 ~ 금 오전 8시 40분 ~ 9시 10분
당신은 선물 111부작 월 ~ 금 저녁 7시 20분 ~ 8시
파도야 파도야 143부작 매주 평일 오전 9:00 ~ 9:40
워킹 맘 육아 대디 120부작 매주 평일 밤 8:55 ~ 9:30
다시 시작해 121부작 평일 저녁 7:15 ~ 7:55
최고의 연인 116부작 매주 평일 저녁 7:15 ~ 7:55
행복을 주는 사람 118부작 평일 저녁 7:15 ~ 7:55
불굴의 차여사 111부작 월 ~ 금 밤 7시 15분 ~ 7시 55분
마녀의 성 122부작 월 ~ 금 저녁 7시 20분 ~ 8시
그 여자의 바다 120부작 월 ~ 금 오전 9시 ~ 9시 40분
훈장 오순남 129부작 월 ~ 금 오전 7시 50분 ~ 8시 30분
꽃 피어라 달순아! 129부작 월 ~ 금 오전 9시 ~ 9시 40분
부잣집 아들 100부작 매주 일요일 밤 8:45 ~ 11:05
언제나 봄날 1

In [224]:
drama_cate['weekday'] = pd.DataFrame(weekday)
drama_cate['weekend'] = pd.DataFrame(weekend)
drama_cate['morning'] = pd.DataFrame(morning)
drama_cate['mini'] = pd.DataFrame(mini)
drama_cate['etc'] = pd.DataFrame(etc)
drama_cate

,name,country,channel,period,count,time,max_rating,source,weekday,weekend,morning,mini,etc,vote_sum
0,시간이 멈추는 그때,대한민국,KBS W,2018년 10월 24일 ~ 2018년 11월 29일,12부작,"수요일, 목요일 밤 11시",0.10%,닐슨코리아,0,0,0,1,0,1
1,그남자 오수,대한민국,OCN,2018년 3월 5일 ~ 2018년 4월 24일,16부작,"월요일, 화요일 밤 9시",0.40%,닐슨코리아,0,0,0,1,0,1
2,애타는 로맨스,대한민국,"OCN, 라이프타임",2017년 4월 17일 ~ 2017년 5월 30일\n2019년 5월 15일 ~ 20...,13부작\n14부작(일본DVD 편집본),"월요일, 화요일 밤 9시\n수요일, 목요일 오후 3시",0.50%,닐슨코리아,0,0,0,1,0,1
3,열두밤,대한민국,채널A,2018년 10월 5일 (스페셜 방송)\n2018년 10월 12일 ~ 2018년 1...,12부작,금요일 밤 11시,0.60%,닐슨코리아,0,0,0,0,0,0
4,커피야 부탁해,대한민국,채널A,2018년 12월 1일 ~ 2018년 12월 30일,12부작,"토요일, 일요일 오후 7시 40분 ~",0.60%,닐슨코리아,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,처음이라서,대한민국,"On Style, OLIVE",2015년 10월 7일 ~2015년 11월 25일,8부작,매주 수요일 밤 11시,조회불가,닐슨코리아,0,0,0,1,0,1
495,파수꾼,대한민국,MBC,2017년 5월 22일 ~ 2017년 7월 11일,32부작,"매주 월, 화 밤 10:00 ~ 11:00",10.20%,닐슨코리아,0,0,0,0,1,1
496,프리스트,대한민국,OCN,2018년 11월 24일 ~ 2019년 1월 20일,16부작,"토요일, 일요일 밤 10시 20분",2.50%,닐슨코리아,0,1,0,0,0,1
497,플레이어,대한민국,OCN,2018년 9월 29일 ~ 2018년 11월 11일,14부작,"토요일, 일요일 밤 10시 20분",5.80%,닐슨코리아,0,1,0,0,0,1


## 3. voting based on results

In [225]:
vote_sum = []
for i in range(len(name)):
    vote_sum.append(0)

In [226]:
for i in range(len(name)):
    print(name[i], ">> ", weekday[i]+weekend[i]+morning[i]+mini[i]+etc[i])
    vote_sum[i] = weekday[i]+weekend[i]+morning[i]+mini[i]+etc[i]

시간이 멈추는 그때 >>  1
그남자 오수 >>  1
애타는 로맨스 >>  1
열두밤 >>  0
커피야 부탁해 >>  1
로스타임 라이프 : 더 라스트 찬스 >>  1
유별나! 문셰프 >>  1
모두의 연애 >>  0
터치  >>  1
달리는 조사관 >>  1
뱀파이어 탐정 >>  1
최고의 치킨 >>  1
시를 잊은 그대에게 >>  1
야식남녀 >>  1
미스트리스  >>  1
솔로몬의 위증 >>  1
조선생존기 >>  1
풍선껌 >>  1
힙합왕 - 나스나길 >>  0
그녀는 거짓말을 너무 사랑해 >>  1
대장금이 보고 있다 >>  1
동네의 영웅 >>  1
멜로가 체질 >>  1
사랑하는 은동아 >>  1
리치맨 >>  1
어서와 >>  1
어쩌다 가족 >>  1
멈추고 싶은 순간 : 어바웃 타임 >>  1
으라차차 와이키키 >>  1
으라차차 와이키키 2 >>  1
평일 오후 세시의 연인 >>  1
라스트 >>  1
마성의 기쁨 >>  1
모두의 거짓말 >>  1
빅 포레스트 >>  0
송곳 >>  1
레버리지: 사기조작단 >>  1
마녀의 사랑 >>  1
안투라지  >>  1
그냥 사랑하는 사이 >>  1
더 패키지 >>  1
반의 반 >>  1
미치겠다, 너땜에! >>  1
신분을 숨겨라 >>  1
마이 리틀 베이비 >>  0
빙의 >>  1
울지 않는 새 >>  1
계약우정  >>  1
날씨가 좋으면 찾아가겠어요 >>  1
판타스틱 >>  1
사이코메트리 그녀석 >>  1
생일편지 >>  1
설렘주의보 >>  1
시카고 타자기 >>  1
안단테 >>  1
킬잇 >>  1
강덕순 애정 변천사 >>  1
써클: 이어진 두 세계 >>  1
17세의 조건 >>  1
마녀보감 >>  1
식샤를 합시다 2 >>  1
싸이코패스 다이어리 >>  1
오 마이 베이비 >>  1
그녀의 사생활 >>  1
맨홀 - 이상한 나라의 필 >>  1
아르곤  >>  1
악마가 너의 이름을 부를 때 >>  1
톱스타 유백이 >>  0
날 녹여주오 >>  1
내성적인 보스 >

In [227]:
drama_cate['vote_sum'] = pd.DataFrame(vote_sum)
drama_cate

,name,country,channel,period,count,time,max_rating,source,weekday,weekend,morning,mini,etc,vote_sum
0,시간이 멈추는 그때,대한민국,KBS W,2018년 10월 24일 ~ 2018년 11월 29일,12부작,"수요일, 목요일 밤 11시",0.10%,닐슨코리아,0,0,0,1,0,1
1,그남자 오수,대한민국,OCN,2018년 3월 5일 ~ 2018년 4월 24일,16부작,"월요일, 화요일 밤 9시",0.40%,닐슨코리아,0,0,0,1,0,1
2,애타는 로맨스,대한민국,"OCN, 라이프타임",2017년 4월 17일 ~ 2017년 5월 30일\n2019년 5월 15일 ~ 20...,13부작\n14부작(일본DVD 편집본),"월요일, 화요일 밤 9시\n수요일, 목요일 오후 3시",0.50%,닐슨코리아,0,0,0,1,0,1
3,열두밤,대한민국,채널A,2018년 10월 5일 (스페셜 방송)\n2018년 10월 12일 ~ 2018년 1...,12부작,금요일 밤 11시,0.60%,닐슨코리아,0,0,0,0,0,0
4,커피야 부탁해,대한민국,채널A,2018년 12월 1일 ~ 2018년 12월 30일,12부작,"토요일, 일요일 오후 7시 40분 ~",0.60%,닐슨코리아,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,처음이라서,대한민국,"On Style, OLIVE",2015년 10월 7일 ~2015년 11월 25일,8부작,매주 수요일 밤 11시,조회불가,닐슨코리아,0,0,0,1,0,1
495,파수꾼,대한민국,MBC,2017년 5월 22일 ~ 2017년 7월 11일,32부작,"매주 월, 화 밤 10:00 ~ 11:00",10.20%,닐슨코리아,0,0,0,0,1,1
496,프리스트,대한민국,OCN,2018년 11월 24일 ~ 2019년 1월 20일,16부작,"토요일, 일요일 밤 10시 20분",2.50%,닐슨코리아,0,1,0,0,0,1
497,플레이어,대한민국,OCN,2018년 9월 29일 ~ 2018년 11월 11일,14부작,"토요일, 일요일 밤 10시 20분",5.80%,닐슨코리아,0,1,0,0,0,1


In [228]:
drama_cate['vote_sum'].value_counts()

1    472
2     13
0     13
3      1
Name: vote_sum, dtype: int64

In [230]:
drama_cate[drama_cate['vote_sum'] == 0].to_csv("not_voting_list.csv", encoding='utf-8-sig', index=False)

## 4. Categorizing based on voting results

In [241]:
category = []
for i in range(len(name)):
    category.append("")

In [252]:
for i in range(len(name)):
    if vote_sum[i] == 1:
        if weekday[i] == 1:
            category[i] = "일일드라마"
        elif weekend[i] == 1:
            category[i] = "주말드라마"
        elif mini[i] == 1:
            category[i] = "미니시리즈"
        elif morning[i] == 1:
            category[i] = "아침드라마"
        elif etc[i] == 1:
            category[i] = "기타"
        else:
            category[i] = ""


In [253]:
drama_cate['category'] = pd.DataFrame(category)

In [254]:
drama_cate

,name,country,channel,period,count,time,max_rating,source,weekday,weekend,morning,mini,etc,vote_sum,category
0,시간이 멈추는 그때,대한민국,KBS W,2018년 10월 24일 ~ 2018년 11월 29일,12부작,"수요일, 목요일 밤 11시",0.10%,닐슨코리아,0,0,0,1,0,1,미니시리즈
1,그남자 오수,대한민국,OCN,2018년 3월 5일 ~ 2018년 4월 24일,16부작,"월요일, 화요일 밤 9시",0.40%,닐슨코리아,0,0,0,1,0,1,미니시리즈
2,애타는 로맨스,대한민국,"OCN, 라이프타임",2017년 4월 17일 ~ 2017년 5월 30일\n2019년 5월 15일 ~ 20...,13부작\n14부작(일본DVD 편집본),"월요일, 화요일 밤 9시\n수요일, 목요일 오후 3시",0.50%,닐슨코리아,0,0,0,1,0,1,미니시리즈
3,열두밤,대한민국,채널A,2018년 10월 5일 (스페셜 방송)\n2018년 10월 12일 ~ 2018년 1...,12부작,금요일 밤 11시,0.60%,닐슨코리아,0,0,0,0,0,0,
4,커피야 부탁해,대한민국,채널A,2018년 12월 1일 ~ 2018년 12월 30일,12부작,"토요일, 일요일 오후 7시 40분 ~",0.60%,닐슨코리아,0,1,0,0,0,1,주말드라마
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,처음이라서,대한민국,"On Style, OLIVE",2015년 10월 7일 ~2015년 11월 25일,8부작,매주 수요일 밤 11시,조회불가,닐슨코리아,0,0,0,1,0,1,미니시리즈
495,파수꾼,대한민국,MBC,2017년 5월 22일 ~ 2017년 7월 11일,32부작,"매주 월, 화 밤 10:00 ~ 11:00",10.20%,닐슨코리아,0,0,0,0,1,1,기타
496,프리스트,대한민국,OCN,2018년 11월 24일 ~ 2019년 1월 20일,16부작,"토요일, 일요일 밤 10시 20분",2.50%,닐슨코리아,0,1,0,0,0,1,주말드라마
497,플레이어,대한민국,OCN,2018년 9월 29일 ~ 2018년 11월 11일,14부작,"토요일, 일요일 밤 10시 20분",5.80%,닐슨코리아,0,1,0,0,0,1,주말드라마


## 5. Data save

In [255]:
drama_cate.to_csv("drama_category_assignment.csv", encoding='utf-8-sig', index=False)

<hr>

# Max-rating을 바탕으로 카테고리별 TOP20 드라마 추출

## 1. Data load

In [313]:
drama4 = pd.read_csv("drama_category_assignment_2.csv")

In [314]:
rating = []
max_rating = drama4['max_rating']
for i in range(len(drama4)):
    rating.append(0)

In [315]:
for i in range(len(drama4)):
    if max_rating[i] == "조회불가":
        rating[i] = -1.0
    else:
        rating[i] = max_rating[i][:max_rating[i].find("%")]

In [316]:
rating = list(map(float, rating))

In [317]:
drama4['rating'] = pd.DataFrame(rating)

In [318]:
drama4

,name,country,channel,period,count,time,max_rating,source,category,note,rating
0,시간이 멈추는 그때,대한민국,KBS W,2018년 10월 24일 ~ 2018년 11월 29일,12부작,"수요일, 목요일 밤 11시",0.10%,닐슨코리아,미니시리즈,NaN,0.10
1,그남자 오수,대한민국,OCN,2018년 3월 5일 ~ 2018년 4월 24일,16부작,"월요일, 화요일 밤 9시",0.40%,닐슨코리아,미니시리즈,NaN,0.40
2,애타는 로맨스,대한민국,"OCN, 라이프타임",2017년 4월 17일 ~ 2017년 5월 30일\n2019년 5월 15일 ~ 20...,13부작\n14부작(일본DVD 편집본),"월요일, 화요일 밤 9시\n수요일, 목요일 오후 3시",0.50%,닐슨코리아,미니시리즈,NaN,0.50
3,열두밤,대한민국,채널A,2018년 10월 5일 (스페셜 방송)\n2018년 10월 12일 ~ 2018년 1...,12부작,금요일 밤 11시,0.60%,닐슨코리아,미니시리즈,NaN,0.60
4,커피야 부탁해,대한민국,채널A,2018년 12월 1일 ~ 2018년 12월 30일,12부작,"토요일, 일요일 오후 7시 40분 ~",0.60%,닐슨코리아,주말드라마,NaN,0.60
...,...,...,...,...,...,...,...,...,...,...,...
494,처음이라서,대한민국,"On Style, OLIVE",2015년 10월 7일 ~2015년 11월 25일,8부작,매주 수요일 밤 11시,조회불가,닐슨코리아,미니시리즈,NaN,-1.00
495,파수꾼,대한민국,MBC,2017년 5월 22일 ~ 2017년 7월 11일,32부작,"매주 월, 화 밤 10:00 ~ 11:00",10.20%,닐슨코리아,미니시리즈,NaN,10.20
496,프리스트,대한민국,OCN,2018년 11월 24일 ~ 2019년 1월 20일,16부작,"토요일, 일요일 밤 10시 20분",2.50%,닐슨코리아,주말드라마,NaN,2.50
497,플레이어,대한민국,OCN,2018년 9월 29일 ~ 2018년 11월 11일,14부작,"토요일, 일요일 밤 10시 20분",5.80%,닐슨코리아,주말드라마,NaN,5.80


In [319]:
drama4['category'].value_counts()

미니시리즈    258
주말드라마     98
기타        74
일일드라마     43
아침드라마     26
Name: category, dtype: int64

In [320]:
drama_mini = drama4[drama4['category'] == '미니시리즈']
drama_morning = drama4[drama4['category'] == '아침드라마']
drama_weekday = drama4[drama4['category'] == '일일드라마']
drama_weekend = drama4[drama4['category'] == '주말드라마']

In [321]:
print(len(drama_mini), len(drama_morning), len(drama_weekday), len(drama_weekend))

258 26 43 98


In [322]:
drama_mini = drama_mini.sort_values(['rating'], ascending=False)
drama_morning = drama_morning.sort_values(['rating'], ascending=False)
drama_weekday = drama_weekday.sort_values(['rating'], ascending=False)
drama_weekend = drama_weekend.sort_values(['rating'], ascending=False)

In [323]:
drama_mini

,name,country,channel,period,count,time,max_rating,source,category,note,rating
468,태양의 후예,대한민국,KBS 2TV,2016년 2월 24일 ~ 2016년 4월 14일,16부작 + 스페셜 3부작,"수, 목",38.80%,닐슨코리아,미니시리즈,NaN,38.8
455,피고인,대한민국,SBS,2017년 1월 23일 ~ 2017년 3월 21일,18부작,"매주 월, 화 밤 10시 ~ 11시 10분",28.30%,닐슨코리아,미니시리즈,NaN,28.3
451,낭만닥터 김사부,대한민국,SBS,2016년 11월 7일 ~ 2017년 1월 17일,20부작 + 스페셜 번외편 1부작,"매주 월, 화 밤 10시 ~ 11시",27.60%,닐슨코리아,미니시리즈,NaN,27.6
452,낭만닥터 김사부 2,대한민국,SBS TV,2020년 1월 6일 ~ 2020년 2월 25일,16부작,"매주 월, 화 밤 9:40 ~ 11시",27.10%,닐슨코리아,미니시리즈,NaN,27.1
440,동백꽃 필 무렵,대한민국,KBS 2TV,2019년 9월 18일 ~ 2019년 11월 21일,40부작 + 스페셜 4부작,"매주 수요일, 목요일 밤 10:00 ~ 11:10",23.80%,닐슨코리아,미니시리즈,NaN,23.8
...,...,...,...,...,...,...,...,...,...,...,...
477,모민의 방,대한민국,OCN,2016년 11월 1일 ~ 2016년 11월 22일,7부작,"매주 화, 목요일 오후 1시",조회불가,닐슨코리아,미니시리즈,NaN,-1.0
481,산후조리원,대한민국,tvN,2020년 11월 ~,8부작,"매주 월, 화 밤 9시",조회불가,닐슨코리아,미니시리즈,NaN,-1.0
489,애간장,대한민국,OCN,2018년 1월 8일 ~ 2018년 2월 6일,10부작,"월요일, 화요일 밤 9시",조회불가,닐슨코리아,미니시리즈,NaN,-1.0
492,유일랍미,대한민국,"드라마 H, TRENDY",2015년 10월 29일 ~ 2015년 12월 18일,16부작,목요일 금요일 낮 12시 30분,조회불가,닐슨코리아,미니시리즈,NaN,-1.0


In [324]:
drama_morning

,name,country,channel,period,count,time,max_rating,source,category,note,rating
396,어머님은 내 며느리,대한민국,SBS,2015년 6월 22일 ~ 2015년 12월 31일,136부작[1],월 ~ 금 오전 8시 30분 ~ 9시 10분,16.90%,닐슨코리아,아침드라마,NaN,16.9
394,내 마음의 꽃비,대한민국,KBS 2TV,2016년 2월 29일 ~ 2016년 9월 6일,128부작,월 ~ 금 오전 9시 ~ 9시 40분,16.40%,닐슨코리아,아침드라마,NaN,16.4
387,별이 되어 빛나리,대한민국,KBS 2TV,2015년 8월 31일 ~ 2016년 2월 26일,128부작,월 ~ 금 오전 9시 ~ 9시 40분,15.90%,닐슨코리아,아침드라마,NaN,15.9
382,이브의 사랑,대한민국,MBC,2015년 5월 18일 ~ 2015년 10월 30일,120부작,월 ~ 금 오전 7시 50분 ~ 8시 30분,14.70%,닐슨코리아,아침드라마,NaN,14.7
381,사랑이 오네요,대한민국,SBS,2016년 6월 20일 ~ 2016년 12월 16일,122부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,14.70%,닐슨코리아,아침드라마,NaN,14.7
378,아임 쏘리 강남구,대한민국,SBS,2016년 12월 19일 ~ 2017년 6월 9일,120부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,14.50%,닐슨코리아,아침드라마,NaN,14.5
361,달콤한 원수,대한민국,SBS,2017년 6월 12일 ~ 2017년 12월 1일,124부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,13.60%,닐슨코리아,아침드라마,NaN,13.6
359,내 사위의 여자,대한민국,SBS,2016년 1월 4일~ 2016년 6월 17일,120부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,13.50%,닐슨코리아,아침드라마,NaN,13.5
352,좋은 사람,대한민국,MBC,2016년 5월 2일 ~ 2016년 10월 28일,122부작,월 ~ 금 오전 7시 50분 ~ 8시 30분,13.00%,닐슨코리아,아침드라마,NaN,13.0
345,그래도 푸르른 날에,대한민국,KBS 2TV,2015년 3월 2일 ~ 2015년 8월 28일,129부작,월 ~ 금 오전 9시 ~ 9시 40분,12.70%,닐슨코리아,아침드라마,NaN,12.7


In [325]:
drama_weekday

,name,country,channel,period,count,time,max_rating,source,category,note,rating
458,빛나라 은수,대한민국,KBS 1TV,2016년 11월 28일 ~ 2017년 5월 26일,125부작,매주 평일 저녁 8:25 ~ 9:00,30.20%,닐슨코리아,일일드라마,NaN,30.2
457,우리집 꿀단지,대한민국,KBS 1TV,2015년 11월 2일 ~ 2016년 4월 29일,129부작,매주 평일 저녁 8:25 ~ 9:00,30.00%,닐슨코리아,일일드라마,NaN,30.0
454,가족을 지켜라,대한민국,KBS 1TV,2015년 5월 11일 ~ 2015년 10월 30일,123부작,매주 평일 저녁 8:25 ~ 9:00,28.20%,닐슨코리아,일일드라마,NaN,28.2
449,별난 가족,대한민국,KBS 1TV,2016년 5월 2일 ~ 2016년 11월 25일,149부작,매주 평일 저녁 8:25 ~ 9:00,26.00%,닐슨코리아,일일드라마,NaN,26.0
447,여름아 부탁해,대한민국,KBS 1TV (본방송)\nKBS 2TV (재방송),2019년 4월 29일 ~ 2019년 10월 25일,128부작,매주 평일 저녁 8:30 ~ 9:00,25.20%,닐슨코리아,일일드라마,NaN,25.2
445,내일도 맑음,대한민국,KBS 1TV,2018년 5월 7일 ~ 2018년 11월 2일,121부작,매주 평일 저녁 8:25 ~ 9:00,24.10%,닐슨코리아,일일드라마,NaN,24.1
442,꽃길만 걸어요,대한민국,KBS 1TV,2019년 10월 28일 ~ 2020년 4월 17일,123부작,매주 평일 저녁 8:30 ~ 9:00,23.90%,닐슨코리아,일일드라마,NaN,23.9
441,이름 없는 여자,대한민국,KBS 2TV,2017년 4월 24일 ~ 2017년 9월 15일,102부작,매주 평일 저녁 7:50 ~ 8:30,23.80%,닐슨코리아,일일드라마,NaN,23.8
438,"다시, 첫사랑",대한민국,KBS 2TV,2016년 11월 28일 ~ 2017년 4월 21일,104부작,매주 평일 저녁 7:50 ~ 8:30,23.70%,닐슨코리아,일일드라마,NaN,23.7
437,무궁화 꽃이 피었습니다,대한민국,KBS 1TV,2017년 5월 29일 ~ 2017년 11월 10일,120부작,매주 평일 저녁 8:25 ~ 9:00,23.40%,닐슨코리아,일일드라마,NaN,23.4


In [326]:
drama_weekend

,name,country,channel,period,count,time,max_rating,source,category,note,rating
470,하나뿐인 내편,대한민국,KBS 2TV,2018년 9월 15일 ~ 2019년 3월 17일,106부작 + 스페셜 2부작,주말 저녁 7시 55분 ~ 밤 9시 15분,49.40%,닐슨코리아,주말드라마,NaN,49.4
469,황금빛 내 인생,대한민국,KBS 2TV,2017년 9월 2일 ~ 2018년 3월 11일,52부작 + 스페셜 2부작,주말 저녁 7시 55분 ~ 밤 9시 15분,45.10%,닐슨코리아,주말드라마,NaN,45.1
467,"부탁해요, 엄마",대한민국,KBS 2TV,2015년 8월 15일 ~ 2016년 2월 14일,54부작,"토, 일 저녁 7시 55분 ~ 밤 9시 15분",38.20%,닐슨코리아,주말드라마,NaN,38.2
466,같이 살래요,대한민국,KBS 2TV,2018년 3월 17일 ~ 2018년 9월 9일,50부작 + 스페셜 1부작,주말 저녁 7시 55분 ~ 밤 9시 15분,36.90%,닐슨코리아,주말드라마,NaN,36.9
465,아버지가 이상해,대한민국,KBS 2TV,2017년 3월 4일 ~ 2017년 8월 27일,52부작,주말 저녁 7시 55분 ~ 밤 9시 15분,36.50%,닐슨코리아,주말드라마,NaN,36.5
...,...,...,...,...,...,...,...,...,...,...,...
21,동네의 영웅,대한민국,OCN,2016년 1월 23일 ~ 2016년 3월 20일,16부작,"토요일, 일요일 밤 11시",1.80%,닐슨코리아,주말드라마,NaN,1.8
16,조선생존기,대한민국,TV CHOSUN,2019년 6월 8일 ~ 2019년 8월 17일,16부작,"매주 토요일, 일요일 밤 10시 50분",1.70%,닐슨코리아,주말드라마,NaN,1.7
14,미스트리스,대한민국,OCN,2018년 4월 28일 ~ 2018년 6월 3일,12부작,"매주 토요일, 일요일 밤 10시",1.60%,닐슨코리아,주말드라마,NaN,1.6
10,뱀파이어 탐정,대한민국,OCN,2016년 3월 27일 ~ 2016년 6월 12일,12부작,일요일 밤 11시,1.30%,닐슨코리아,주말드라마,NaN,1.3


# 2. Select TOP20!

In [327]:
drama_mini_top20 = drama_mini[:20]
drama_mini_top20.reset_index(inplace=True)
drama_mini_top20

,index,name,country,channel,period,count,time,max_rating,source,category,note,rating
0,468,태양의 후예,대한민국,KBS 2TV,2016년 2월 24일 ~ 2016년 4월 14일,16부작 + 스페셜 3부작,"수, 목",38.80%,닐슨코리아,미니시리즈,NaN,38.8
1,455,피고인,대한민국,SBS,2017년 1월 23일 ~ 2017년 3월 21일,18부작,"매주 월, 화 밤 10시 ~ 11시 10분",28.30%,닐슨코리아,미니시리즈,NaN,28.3
2,451,낭만닥터 김사부,대한민국,SBS,2016년 11월 7일 ~ 2017년 1월 17일,20부작 + 스페셜 번외편 1부작,"매주 월, 화 밤 10시 ~ 11시",27.60%,닐슨코리아,미니시리즈,NaN,27.6
3,452,낭만닥터 김사부 2,대한민국,SBS TV,2020년 1월 6일 ~ 2020년 2월 25일,16부작,"매주 월, 화 밤 9:40 ~ 11시",27.10%,닐슨코리아,미니시리즈,NaN,27.1
4,440,동백꽃 필 무렵,대한민국,KBS 2TV,2019년 9월 18일 ~ 2019년 11월 21일,40부작 + 스페셜 4부작,"매주 수요일, 목요일 밤 10:00 ~ 11:10",23.80%,닐슨코리아,미니시리즈,NaN,23.8
5,436,구르미 그린 달빛,대한민국,KBS 2TV,2016년 8월 22일 ~ 2016년 10월 18일,18부작\n스페셜(특별판 3부작 + 별전 1부작),"매주 월요일, 화요일 밤 10:00 ~ 11:00",23.30%,닐슨코리아,미니시리즈,NaN,23.3
6,433,왜그래 풍상씨,대한민국,KBS 2TV,2019년 1월 9일 ~ 2019년 3월 14일,40부작 + 스페셜 1부작,"매주 수요일, 목요일 밤 10:00 ~ 11:10",22.70%,닐슨코리아,미니시리즈,NaN,22.7
7,425,용팔이,대한민국,SBS,2015년 8월 5일 ~ 2015년 10월 1일,18부작[1],"수요일, 목요일 밤 10시 ~ 11시 15분",21.50%,닐슨코리아,미니시리즈,NaN,21.5
8,423,닥터스,대한민국,SBS,2016년 6월 20일 ~ 2016년 8월 23일,20부작,"월, 화 밤 10시 ~ 11시 10분",21.30%,닐슨코리아,미니시리즈,NaN,21.3
9,422,푸른 바다의 전설,대한민국,SBS TV,2016년 11월 16일 ~ 2017년 1월 25일,20부작 + 스페셜 1부작,"매주 수, 목 밤 10:00 ~ 11:10",21.00%,닐슨코리아,미니시리즈,NaN,21.0


In [328]:
drama_morning_top20 = drama_morning[:20]
drama_morning_top20.reset_index(inplace=True)
drama_morning_top20

,index,name,country,channel,period,count,time,max_rating,source,category,note,rating
0,396,어머님은 내 며느리,대한민국,SBS,2015년 6월 22일 ~ 2015년 12월 31일,136부작[1],월 ~ 금 오전 8시 30분 ~ 9시 10분,16.90%,닐슨코리아,아침드라마,NaN,16.9
1,394,내 마음의 꽃비,대한민국,KBS 2TV,2016년 2월 29일 ~ 2016년 9월 6일,128부작,월 ~ 금 오전 9시 ~ 9시 40분,16.40%,닐슨코리아,아침드라마,NaN,16.4
2,387,별이 되어 빛나리,대한민국,KBS 2TV,2015년 8월 31일 ~ 2016년 2월 26일,128부작,월 ~ 금 오전 9시 ~ 9시 40분,15.90%,닐슨코리아,아침드라마,NaN,15.9
3,382,이브의 사랑,대한민국,MBC,2015년 5월 18일 ~ 2015년 10월 30일,120부작,월 ~ 금 오전 7시 50분 ~ 8시 30분,14.70%,닐슨코리아,아침드라마,NaN,14.7
4,381,사랑이 오네요,대한민국,SBS,2016년 6월 20일 ~ 2016년 12월 16일,122부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,14.70%,닐슨코리아,아침드라마,NaN,14.7
5,378,아임 쏘리 강남구,대한민국,SBS,2016년 12월 19일 ~ 2017년 6월 9일,120부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,14.50%,닐슨코리아,아침드라마,NaN,14.5
6,361,달콤한 원수,대한민국,SBS,2017년 6월 12일 ~ 2017년 12월 1일,124부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,13.60%,닐슨코리아,아침드라마,NaN,13.6
7,359,내 사위의 여자,대한민국,SBS,2016년 1월 4일~ 2016년 6월 17일,120부작,월 ~ 금 오전 8시 30분 ~ 9시 10분,13.50%,닐슨코리아,아침드라마,NaN,13.5
8,352,좋은 사람,대한민국,MBC,2016년 5월 2일 ~ 2016년 10월 28일,122부작,월 ~ 금 오전 7시 50분 ~ 8시 30분,13.00%,닐슨코리아,아침드라마,NaN,13.0
9,345,그래도 푸르른 날에,대한민국,KBS 2TV,2015년 3월 2일 ~ 2015년 8월 28일,129부작,월 ~ 금 오전 9시 ~ 9시 40분,12.70%,닐슨코리아,아침드라마,NaN,12.7


In [329]:
drama_weekend_top20 = drama_weekend[:20]
drama_weekend_top20.reset_index(inplace=True)
drama_weekend_top20

,index,name,country,channel,period,count,time,max_rating,source,category,note,rating
0,470,하나뿐인 내편,대한민국,KBS 2TV,2018년 9월 15일 ~ 2019년 3월 17일,106부작 + 스페셜 2부작,주말 저녁 7시 55분 ~ 밤 9시 15분,49.40%,닐슨코리아,주말드라마,NaN,49.4
1,469,황금빛 내 인생,대한민국,KBS 2TV,2017년 9월 2일 ~ 2018년 3월 11일,52부작 + 스페셜 2부작,주말 저녁 7시 55분 ~ 밤 9시 15분,45.10%,닐슨코리아,주말드라마,NaN,45.1
2,467,"부탁해요, 엄마",대한민국,KBS 2TV,2015년 8월 15일 ~ 2016년 2월 14일,54부작,"토, 일 저녁 7시 55분 ~ 밤 9시 15분",38.20%,닐슨코리아,주말드라마,NaN,38.2
3,466,같이 살래요,대한민국,KBS 2TV,2018년 3월 17일 ~ 2018년 9월 9일,50부작 + 스페셜 1부작,주말 저녁 7시 55분 ~ 밤 9시 15분,36.90%,닐슨코리아,주말드라마,NaN,36.9
4,465,아버지가 이상해,대한민국,KBS 2TV,2017년 3월 4일 ~ 2017년 8월 27일,52부작,주말 저녁 7시 55분 ~ 밤 9시 15분,36.50%,닐슨코리아,주말드라마,NaN,36.5
5,464,월계수 양복점 신사들,대한민국,KBS 2TV,2016년 8월 27일 ~ 2017년 2월 26일,54부작,주말 저녁 7시 55분 ~ 밤 9시 15분,36.20%,닐슨코리아,주말드라마,NaN,36.2
6,463,세상에서 제일 예쁜 내 딸,대한민국,KBS 2TV,2019년 3월 23일 ~ 2019년 9월 22일,108부작,주말 토요일~일요일 저녁 7시 55분 ~ 밤 9시 15분,35.90%,닐슨코리아,주말드라마,NaN,35.9
7,462,"내 딸, 금사월",대한민국,MBC,2015년 9월 5일 ~ 2016년 2월 28일,51부작,"토, 일 밤 10시 ~ 11시 10분",34.90%,닐슨코리아,주말드라마,NaN,34.9
8,461,아이가 다섯,대한민국,KBS 2TV,2016년 2월 20일 ~ 2016년 8월 21일,54부작,주말 저녁 7시 55분 ~ 밤 9시 15분,32.80%,닐슨코리아,주말드라마,NaN,32.8
9,460,사랑은 뷰티풀 인생은 원더풀,대한민국,KBS 2TV,2019년 9월 28일 ~ 2020년 3월 22일,100부작 + 스페셜 4부작,매주 주말 저녁 7:55 ~ 밤 9:15,32.30%,닐슨코리아,주말드라마,NaN,32.3


In [330]:
drama_weekday_top20 = drama_weekday[:20]
drama_weekday_top20.reset_index(inplace=True)
drama_weekday_top20

,index,name,country,channel,period,count,time,max_rating,source,category,note,rating
0,458,빛나라 은수,대한민국,KBS 1TV,2016년 11월 28일 ~ 2017년 5월 26일,125부작,매주 평일 저녁 8:25 ~ 9:00,30.20%,닐슨코리아,일일드라마,NaN,30.2
1,457,우리집 꿀단지,대한민국,KBS 1TV,2015년 11월 2일 ~ 2016년 4월 29일,129부작,매주 평일 저녁 8:25 ~ 9:00,30.00%,닐슨코리아,일일드라마,NaN,30.0
2,454,가족을 지켜라,대한민국,KBS 1TV,2015년 5월 11일 ~ 2015년 10월 30일,123부작,매주 평일 저녁 8:25 ~ 9:00,28.20%,닐슨코리아,일일드라마,NaN,28.2
3,449,별난 가족,대한민국,KBS 1TV,2016년 5월 2일 ~ 2016년 11월 25일,149부작,매주 평일 저녁 8:25 ~ 9:00,26.00%,닐슨코리아,일일드라마,NaN,26.0
4,447,여름아 부탁해,대한민국,KBS 1TV (본방송)\nKBS 2TV (재방송),2019년 4월 29일 ~ 2019년 10월 25일,128부작,매주 평일 저녁 8:30 ~ 9:00,25.20%,닐슨코리아,일일드라마,NaN,25.2
5,445,내일도 맑음,대한민국,KBS 1TV,2018년 5월 7일 ~ 2018년 11월 2일,121부작,매주 평일 저녁 8:25 ~ 9:00,24.10%,닐슨코리아,일일드라마,NaN,24.1
6,442,꽃길만 걸어요,대한민국,KBS 1TV,2019년 10월 28일 ~ 2020년 4월 17일,123부작,매주 평일 저녁 8:30 ~ 9:00,23.90%,닐슨코리아,일일드라마,NaN,23.9
7,441,이름 없는 여자,대한민국,KBS 2TV,2017년 4월 24일 ~ 2017년 9월 15일,102부작,매주 평일 저녁 7:50 ~ 8:30,23.80%,닐슨코리아,일일드라마,NaN,23.8
8,438,"다시, 첫사랑",대한민국,KBS 2TV,2016년 11월 28일 ~ 2017년 4월 21일,104부작,매주 평일 저녁 7:50 ~ 8:30,23.70%,닐슨코리아,일일드라마,NaN,23.7
9,437,무궁화 꽃이 피었습니다,대한민국,KBS 1TV,2017년 5월 29일 ~ 2017년 11월 10일,120부작,매주 평일 저녁 8:25 ~ 9:00,23.40%,닐슨코리아,일일드라마,NaN,23.4


## 3. Data save

In [331]:
drama_weekday_top20.to_csv("drama_weekday_top20.csv", index=False, encoding='utf-8-sig')
drama_weekend_top20.to_csv("drama_weekend_top20.csv", index=False, encoding='utf-8-sig')
drama_morning_top20.to_csv("drama_morning_top20.csv", index=False, encoding='utf-8-sig')
drama_mini_top20.to_csv("drama_mini_top20.csv", index=False, encoding='utf-8-sig')